In [1]:
import assets.helper as b3
import assets.functions as run


In [2]:
import pandas as pd
import numpy as np
import os

import assets.graphs as g
import random
import requests

### QUOTES PRICES

In [ ]:
fund = run.sys_load_pkl(f'{b3.app_folder}/fund')
quotes = run.sys_load_pkl(f'{b3.app_folder}/quotes')


In [ ]:
fund.keys(), fund['BENS INDUSTRIAIS'].columns

In [ ]:
quotes.keys(), quotes['AERIS'].keys()

In [ ]:
df_preplot = {}
start_time = run.time.time()

for i, (setor, df_fund) in enumerate(fund.items()):
    df_fund = preprocess_data(df_fund)
    df_cd_conta = pivot_data(df_fund)

    df_unique = df_fund.drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_premerged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])
    df_resampled = resample_data(df_premerged)
    
    df_merged = merge_with_bigdata(df_resampled, bigdata)
    df_merged = df_merged.set_index('Date', drop=True)
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    # Clean up the dataframe using the cleanup_dataframe function
    df_merged = cleanup_dataframe(df_merged)

    df_merged = add_metrics(df_merged)

    df_preplot[setor] = df_merged

    print(setor, run.sys_remaining_time(start_time, len(fund), i))


In [ ]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
start_time = run.time.time()
for i, (setor, df) in enumerate(df_preplot.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.sys_save_pkl(df_temp, f'{b3.app_folder}/{company_folder}/{company}')
        except Exception as e:
            pass
    print(setor, run.sys_remaining_time(start_time, len(df_preplot), i))


In [ ]:
mask = df_preplot['BENS INDUSTRIAIS']['TICKER'] == 'AERI3'
df_preplot['BENS INDUSTRIAIS'][mask]

### DASH

In [ ]:
company = 'AZEVEDO'
df = run.sys_load_pkl(f'{b3.app_folder}/{b3.company_folder}/{company}')
tickers = df['TICKER'].unique()
# df.columns.to_list()

In [ ]:
df['03.11 - Lucro Líquido']
df['14.04.01 - ROE (Resultado por Patrimônio)']
df['03.01 - Receita Bruta']
df['06.01 - Caixa das Operações']





In [ ]:
graphs = g.construct_graphs(df)
graphs.keys()

In [ ]:
import assets.lines
lines = assets.lines.report
groups = set(entry['title'][:2] for entry in lines)
groups

In [ ]:
for g, (group_key, group) in enumerate(graphs.items()):
    status = True if g == 0 else False
    plots = []
    for l, (line_key, line) in enumerate(group.items()):
        for p, (plot_key, plot_info) in enumerate(line.items()):
            for ticker in df['TICKER'].unique():
                df_ticker = df[df['TICKER'] == ticker]
                if not df.empty:
                    print(group[0][0]['info']['title'])
                    print(line[0]['info']['title'])
                    print(plot_info['info']['title'])
                    run.plot_tweak(plot_info, df_ticker).show();

In [ ]:
for ticker in tickers:
    df = df[df['TICKER'] == ticker]
    print(ticker, len(df))

    if not df.empty:
        for g, (group_key, group) in enumerate(merged_entries.items()):
            status = True if g == 0 else False
            plots = []
            for l, (line_key, line) in enumerate(group.items()):
                for p, (plot_key, plot_info) in enumerate(line.items()):
                    print(ticker)
                    run.plot_tweak(plot_info, df).show();

In [ ]:
fund = run.load_database()


In [ ]:
setores = []
for setor, df in fund.items():
    columns=['SETOR', 'SUBSETOR', 'SEGMENTO', 'PREGAO']
    setores.append(df[columns].drop_duplicates())
    print(setor)
sss = pd.concat(setores)
sss = sss.applymap(run.clean_text)

sss = run.sys_save_pkl(sss, f'{b3.app_folder}/sss')

In [ ]:
sss

In [ ]:
mask = sss['SUBSETOR'] == 'MATERIAL DE TRANSPORTE'
sss[mask]

#### Playground


In [ ]:
company_list = run.sys_load_pkl('company_list')
company_list

In [ ]:
listagem_siglas = ['NM', 'N1', 'N2', 'MA', 'M2', 'MB', 'DR1', 'DR2', 'DR3', 'DRE', 'DRN']
listagem_extenso = ['Novo Mercado', 'Nível 1 de Governança Corporativa', 'Nível 2 de Governança Corporativa', 'Bovespa Mais', 'Bovespa Mais Nível 2', 'Organização Tradicional OTC', 'BDR Nível 1', 'BDR Nível 2', 'BDR Nível 3', 'BDR ETF', 'BDR Não Patrocinado']
list_dict = dict(zip(listagem_siglas, listagem_extenso))
list_dict

#### Merge and update


In [ ]:
driver, wait = run.sys_load_browser()
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/search?language=pt-br' 



In [ ]:
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
companies__local = run.sys_read_or_create_dataframe('company', b3_cols).fillna('')
b3_companies_tickers = run.sys_load_pkl('b3_companies_tickers')

b3_companies_tickers = run.get_b3_tickers(driver, wait, url)
for col in b3_cols:
    if col not in b3_companies_tickers.columns:
        b3_companies_tickers[col] = ''
b3_companies_tickers = b3_companies_tickers[b3_cols]
# b3_companies_tickers = sys_save_pkl(b3_companies_tickers, 'b3_companies_tickers')

key_columns = ['ticker', 'company_name']
len(b3_companies_tickers), len(companies__local)

In [ ]:
merged = pd.merge(companies__local, b3_companies_tickers[:], how='outer', on=key_columns, indicator=True)

update_strict = merged[merged['_merge'] == 'right_only'][key_columns] # only companies new in web
update_broad = merged[merged['_merge'] != 'left_only'][key_columns] # all companies from web
updated = update_strict

len(update_broad), len(update_strict)

In [ ]:
key_columns = ['company_name', 'ticker']
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
b3_cols_float = ['Capital Social']
b3_str_cols = [col for col in b3_cols if col not in  b3_cols_float]
col_types = {col: 'float' if col in b3_cols_float else 'str' for col in b3_cols}


In [ ]:
driver.get(b3.url)
# time.sleep(1)

size = len(updated)
# Iterate through the new companies to extract detailed information
new_companies = []
start_time = run.time.time()
for i, (index, row) in enumerate(updated.iterrows()):
    driver.get(b3.url)
    new_company = run.b3_get_company_info(row, driver, wait)
    new_company = pd.DataFrame([new_company], columns=b3_cols).astype(col_types)
    new_company['Capital Social'] = new_company['Capital Social'].replace(np.nan, 0.0)
    new_company = new_company.replace('nan', '')
    new_company['Capital Social'] = pd.to_numeric(new_company['Capital Social'], errors='coerce').astype('float')

    try:
        if not new_company.empty:
            cnpj = new_company['cnpj'][0]
            url = f'https://cnpj.biz/{cnpj}'
            response = requests.get(url, headers={"User-Agent": random.choice(b3.USER_AGENTS)})
            extra = run.b3_get_company_extra_cnpj_info(response)
            extra = pd.DataFrame([extra], columns=b3_cols).fillna('')
            extra = extra.astype(col_types)
        else:
            extra = pd.DataFrame([], columns=b3_cols)

    except Exception as e:
        pass
    extra = extra.reset_index(drop=True)
    new_company = pd.merge(new_company[b3.cols_b3_companies], extra[b3.col_b3_companies_extra_columns], left_on='cnpj', right_on='CNPJ', how='outer').fillna('').reset_index(drop=True)
    new_companies.append(new_company)
    
    print(run.sys_remaining_time(start_time, size, i), row['ticker'], row['company_name'])

    if (size-i-1) % (b3.bin_size/10) == 0:
        temp = pd.concat(new_companies).reset_index(drop=True)
        temp['Capital Social'] = pd.to_numeric(temp['Capital Social'], errors='coerce').astype('float')
        temp['Capital Social'] = temp['Capital Social'].replace(np.nan, 0.0)
        temp = pd.merge(companies__local, temp, on=b3_cols, how='outer', indicator=False).fillna('').drop_duplicates(subset=key_columns, keep='last').reset_index(drop=True)
        temp = sys_save_and_pickle(temp, 'company')


In [ ]:
temp

In [ ]:
for col in b3_cols:
    d1 = new_company[col].dtype
    d2 = extra[col].dtype
    print(col, d1, d2)
    if d1 != d2:
        print(col, d1, d2)
    

In [ ]:
b3_cols

##### CVM GET CODE

In [ ]:
for i in range(0,1000+1):
    url = f'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/{i}/ABCD/overview?language=pt-br'
    print(url)

##### Final Company Get

In [ ]:
companies__local = run.sys_read_or_create_dataframe('company', b3_cols)


In [ ]:
companies__local.columns

In [ ]:
mask = companies__local['Estado'] == ''
# mask &= companies__local['cnpj'] != ''
companies__local = companies__local[~mask]

In [ ]:
companies__local = run.sys_save_and_pickle(companies__local, 'companies__local')

### Playground

In [ ]:
import winsound


In [ ]:
frequency = 5000  # Set Frequency in Hertz
duration = 50   # Set Duration in ms == 1 second
winsound.Beep(frequency, duration)


In [ ]:
cvm_local = run.sys_load_pkl(f'{b3.app_folder}/' + 'cvm')
for year, df in cvm_local.items():
    print(year, df.shape)

In [ ]:
cvm_web = run.sys_load_pkl(f'{b3.app_folder}/temp_' + 'cvm_web_clean')
for year, df in cvm_web.items():
    print(year, df.shape)

In [ ]:
year = 2021
math_local = run.sys_load_pkl(f'{b3.app_folder}/temp_math_local_{year}')
math_web = run.sys_load_pkl(f'{b3.app_folder}/temp_math_web_{year}')
math_local.shape, math_web.shape

In [ ]:
mask = math_local['CNPJ_CIA'] == '04.821.041/0001-08'
mask &= math_local['AGRUPAMENTO'] == 'ind'
mask &= math_local['DT_REFER'] == '2021-12-31'
mask &= math_local['CD_CONTA'] == '5.01'
mask &= math_local['COLUNA_DF'] == 'Capital Social Integralizado'
math_local[mask]
#  [['AGRUPAMENTO', 'CNPJ_CIA', 'DT_REFER', 'CD_CONTA', 'VL_CONTA']]


In [ ]:
mask = math_web['CNPJ_CIA'] == '04.821.041/0001-08'
mask &= math_web['DT_REFER'] == '2021-12-31'
mask &= math_web['CD_CONTA'] == '5.01'
mask &= math_web['AGRUPAMENTO'] == 'ind'
mask &= math_web['COLUNA_DF'] == 'Capital Social Integralizado'
math_web[mask].drop_duplicates()
#  AGRUPAMENTO, CD_CONTA, VL_CONTA

In [ ]:
cvm_local = run.sys_load_pkl(f'{b3.app_folder}/cvm_web')
cvm_local.keys()

In [ ]:
filename = 'acoes'
columns = ['Companhia', 'Trimestre', 'Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria', 'Unidade', 'URL']
acoes = run.sys_read_or_create_dataframe(filename, columns)
acoes.to_csv('acoes.csv')


In [ ]:
filename = 'nsd_links'
cols_nsd = ['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor', 'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd']

nsd = run.sys_read_or_create_dataframe(filename, cols_nsd)
nsd.to_csv('nsd_links.csv')

In [ ]:
filename = 'company'
columns = ['pregao', 'company_name', 'cvm', 'listagem', 'ticker', 'tickers',
       'asin', 'cnpj', 'site', 'setor', 'subsetor', 'segmento', 'atividade',
       'escriturador', 'stock_holders', 'url', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Data da Abertura', 'Porte', 'Natureza Jurídica',
       'Opção pelo MEI', 'Opção pelo Simples', 'Capital Social', 'Tipo',
       'Situação', 'Logradouro', 'Bairro', 'CEP', 'Município', 'Estado']

company = run.sys_read_or_create_dataframe(filename, columns)
company.to_csv('company.csv')

In [ ]:
mask = company['Situação'] == 'Ativa'
mask &= company['setor'] != 'NENHUM'
mask &= company['setor'] != 'NAO CLASSIFICADOS'
mask &= company['setor'] != 'OUTROS'
mask = company['Estado'] == ''
company[mask]


In [ ]:
cvm_local = run.sys_load_pkl(f'{b3.app_folder}/cvm')
cvm_web = run.sys_load_pkl(f'{b3.app_folder}/temp_'+'cvm_updated')


In [ ]:
for year, df in cvm_web.items():
    print(year, df.shape)

In [3]:
cvm_local = run.sys_load_pkl(f'{b3.app_folder}/cvm')
cvm_web = run.sys_load_pkl(f'{b3.app_folder}/temp_' + 'cvm_web_clean')


In [163]:
year = 2019
df_local = cvm_local[year]
df_web = cvm_web[year]
df_columns = df_web.columns

def mask(df):
    mask = df['CNPJ_CIA'] != ''
    mask &= df['CNPJ_CIA'] == '00.000.000/0001-91'
    mask &= (df['CD_CONTA'] == '3.01') | (df['CD_CONTA'] == '5.01')
    mask &= (df['COLUNA_DF'].isna() | (df['COLUNA_DF'] == 'Capital Social Integralizado'))
    mask &= df['AGRUPAMENTO'] == 'ind'
    df = df[mask]
    # mask = ~df['DT_REFER'].dt.quarter.isin([2, 3]) | (df['DT_REFER'].dt.quarter == df['DT_INI_EXERC'].dt.quarter)

    # df = df[mask]
    return df

df_web = mask(df_web)[df_columns][2:]
df_local = mask(df_local)[df_columns][:3]
df_web.to_csv('df_web.csv', index=False)
df_web.to_csv('df_local.csv', index=False)


In [165]:
# df_columns = ['AGRUPAMENTO', 'CNPJ_CIA', 'DT_REFER', 'CD_CONTA', 'DS_CONTA', 'COLUNA_DF', 'VL_CONTA', ]

# Define the list of columns that should not be included as key columns for merging.
compare_columns = ['VL_CONTA']

# Generate the list of key columns for merging by excluding the columns listed in no_columns.
key_columns = [col for col in df_columns if col not in compare_columns]

# updated values
# Merge the local and web DataFrames based on the key columns, and an indicator column 
df_updated_values = pd.merge(df_local, df_web, on=key_columns, how='outer', suffixes=('_local', '_web'), indicator=True)

existing_lines = df_updated_values['_merge'] == 'left_only'  # Rows that exist only in the local DataFrame.
new_lines = df_updated_values['_merge'] == 'right_only'  # Rows that exist only in the web DataFrame.
updated_lines = (df_updated_values['_merge'] == 'both') & (df_updated_values['VL_CONTA_local'] != df_updated_values['VL_CONTA_web'])  # Rows that exist in both but have different 'VL_CONTA' values.

# Define conditions to determine which values to keep in the merged DataFrame.
conditions = [
    existing_lines, 
    new_lines, 
    updated_lines, 
]

# Define the choices corresponding to each condition.
choices = [
    df_updated_values['VL_CONTA_local'],  # existing_lines = Keep the local value for 'VL_CONTA' if the row is from local only.
    df_updated_values['VL_CONTA_web'],    # new_lines = Keep the web value for 'VL_CONTA' if the row is from web only.
    df_updated_values['VL_CONTA_web']     # updated_lines = Keep the web value for 'VL_CONTA' if the values differ between local and web.
]

# Apply the conditions to create a new 'VL_CONTA' column in the merged DataFrame.
df_updated_values['VL_CONTA'] = np.select(conditions, choices, default=df_updated_values['VL_CONTA_web'])

# updated rows (rows with updated values)
# Create a mask to identify rows that have been updated or are new from the web DataFrame.
updated_rows_mask = (new_lines | updated_lines)

# Use the mask to filter the merged DataFrame to only include updated or new rows.
df_updated_rows = df_updated_values[updated_rows_mask]

# updated_quarters
# Define the columns to be used for the final filtering.
quarter_col = ['']
unique_sheet_cols = ['CNPJ_CIA', 'AGRUPAMENTO', 'CD_CONTA', 'DS_CONTA', 'COLUNA_DF']

filter_cols = [col for col in unique_sheet_cols if col not in quarter_col]
df_updated_rows[filter_cols]

# Perform an inner merge to filter the original merged DataFrame using the updated rows 
# and keep only the rows with matching values in the specified filter columns.
df_updated_quarters = pd.merge(
    df_updated_rows[filter_cols].drop_duplicates(),  # Only the columns to match from the updated rows.
    df_updated_values,  # The original merged DataFrame - the updated complete df
    on=filter_cols,  # Columns to match on 
    how='inner'  # Keep only matches found in both DataFrames.
)
df_updated_quarters = df_updated_quarters.sort_values(by=['CNPJ_CIA', 'AGRUPAMENTO', 'CD_CONTA', 'DS_CONTA', 'COLUNA_DF', 'DT_REFER', ])[df_columns]
df_updated_quarters

,FILENAME,DEMONSTRATIVO,BALANCE_SHEET,ANO,AGRUPAMENTO,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
5,itr_cia_aberta_DRE_ind_2019.csv,itr,DRE,2019,ind,00.000.000/0001-91,2019-03-31,1,BCO BRASIL SA,1023,DF Individual - Demonstração do Resultado,REAL,UNIDADE,2019-03-31,3.01,Receitas da Intermediação Financeira,3.167807e+10,S,2019-01-01,NaN
6,itr_cia_aberta_DRE_ind_2019.csv,itr,DRE,2019,ind,00.000.000/0001-91,2019-06-30,1,BCO BRASIL SA,1023,DF Individual - Demonstração do Resultado,REAL,UNIDADE,2019-06-30,3.01,Receitas da Intermediação Financeira,6.301728e+10,S,2019-01-01,NaN
7,itr_cia_aberta_DRE_ind_2019.csv,itr,DRE,2019,ind,00.000.000/0001-91,2019-06-30,1,BCO BRASIL SA,1023,DF Individual - Demonstração do Resultado,REAL,UNIDADE,2019-06-30,3.01,Receitas da Intermediação Financeira,3.131084e+10,S,2019-04-01,NaN
8,itr_cia_aberta_DRE_ind_2019.csv,itr,DRE,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Demonstração do Resultado,REAL,UNIDADE,2019-09-30,3.01,Receitas da Intermediação Financeira,1.005962e+11,S,2019-01-01,NaN
9,itr_cia_aberta_DRE_ind_2019.csv,itr,DRE,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Demonstração do Resultado,REAL,UNIDADE,2019-09-30,3.01,Receitas da Intermediação Financeira,3.757889e+10,S,2019-07-01,NaN
10,dfp_cia_aberta_DRE_ind_2019.csv,dfp,DRE,2019,ind,00.000.000/0001-91,2019-12-31,2,BCO BRASIL SA,1023,DF Individual - Demonstração do Resultado,REAL,UNIDADE,2019-12-31,3.01,Receitas da Intermediação Financeira,1.284906e+11,S,2019-01-01,NaN
0,itr_cia_aberta_DMPL_ind_2019.csv,itr,DMPL,2019,ind,00.000.000/0001-91,2019-03-31,1,BCO BRASIL SA,1023,DF Individual - Demonstração das Mutações do P...,REAL,MIL,2019-03-31,5.01,Saldo Inicial,6.700000e+07,S,2019-01-01,Capital Social Integralizado
1,itr_cia_aberta_DMPL_ind_2019.csv,itr,DMPL,2019,ind,00.000.000/0001-91,2019-06-30,1,BCO BRASIL SA,1023,DF Individual - Demonstração das Mutações do P...,REAL,MIL,2019-06-30,5.01,Saldo Inicial,6.700000e+07,S,2019-01-01,Capital Social Integralizado
2,itr_cia_aberta_DMPL_ind_2019.csv,itr,DMPL,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Demonstração das Mutações do P...,REAL,MIL,2019-09-30,5.01,Saldo Inicial,6.700000e+07,S,2019-01-01,Capital Social Integralizado
3,itr_cia_aberta_DMPL_ind_2019.csv,itr,DMPL,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Demonstração das Mutações do P...,REAL,UNIDADE,2019-09-30,5.01,Saldo Inicial,6.700000e+10,S,2019-01-01,Capital Social Integralizado


In [200]:
df = cvm_local[year]
mask = df['CNPJ_CIA'] != ''
mask &= df['CNPJ_CIA'] == '00.000.000/0001-91'
mask &= ((df['CD_CONTA'] == '1') | (df['CD_CONTA'] == '2') | (df['CD_CONTA'] == '3.01') | (df['CD_CONTA'] == '4.01') | (df['CD_CONTA'] == '5.01') | (df['CD_CONTA'] == '6.01') | (df['CD_CONTA'] == '7.01'))
mask &= df['AGRUPAMENTO'] == 'ind'
df[mask].to_csv('df.csv', index=False)
df[mask]

,FILENAME,DEMONSTRATIVO,BALANCE_SHEET,ANO,AGRUPAMENTO,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
126069,itr_cia_aberta_BPA_ind_2019.csv,itr,BPA,2019,ind,00.000.000/0001-91,2019-03-31,1,BCO BRASIL SA,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,2019-03-31,1,Ativo Total,1.579306e+09,S,NaT,NaN
126265,itr_cia_aberta_BPA_ind_2019.csv,itr,BPA,2019,ind,00.000.000/0001-91,2019-06-30,1,BCO BRASIL SA,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,2019-06-30,1,Ativo Total,1.601798e+09,S,NaT,NaN
126439,itr_cia_aberta_BPA_ind_2019.csv,itr,BPA,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,2019-09-30,1,Ativo Total,1.568536e+09,S,NaT,NaN
575780,itr_cia_aberta_BPP_ind_2019.csv,itr,BPP,2019,ind,00.000.000/0001-91,2019-03-31,1,BCO BRASIL SA,1023,DF Individual - Balanço Patrimonial Passivo,REAL,MIL,2019-03-31,2,Passivo Total,1.579306e+09,S,NaT,NaN
575980,itr_cia_aberta_BPP_ind_2019.csv,itr,BPP,2019,ind,00.000.000/0001-91,2019-06-30,1,BCO BRASIL SA,1023,DF Individual - Balanço Patrimonial Passivo,REAL,MIL,2019-06-30,2,Passivo Total,1.601798e+09,S,NaT,NaN
576170,itr_cia_aberta_BPP_ind_2019.csv,itr,BPP,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Balanço Patrimonial Passivo,REAL,MIL,2019-09-30,2,Passivo Total,1.568536e+09,S,NaT,NaN
1054545,itr_cia_aberta_DFC_MI_ind_2019.csv,itr,DFC_MI,2019,ind,00.000.000/0001-91,2019-03-31,1,BCO BRASIL SA,1023,DF Individual - Demonstração do Fluxo de Caixa...,REAL,MIL,2019-03-31,6.01,Caixa Líquido Atividades Operacionais,3.354647e+07,S,2019-01-01,NaN
1054651,itr_cia_aberta_DFC_MI_ind_2019.csv,itr,DFC_MI,2019,ind,00.000.000/0001-91,2019-06-30,1,BCO BRASIL SA,1023,DF Individual - Demonstração do Fluxo de Caixa...,REAL,MIL,2019-06-30,6.01,Caixa Líquido Atividades Operacionais,4.615182e+07,S,2019-01-01,NaN
1054765,itr_cia_aberta_DFC_MI_ind_2019.csv,itr,DFC_MI,2019,ind,00.000.000/0001-91,2019-09-30,2,BCO BRASIL SA,1023,DF Individual - Demonstração do Fluxo de Caixa...,REAL,MIL,2019-09-30,6.01,Caixa Líquido Atividades Operacionais,5.753469e+07,S,2019-01-01,NaN
1638864,itr_cia_aberta_DMPL_ind_2019.csv,itr,DMPL,2019,ind,00.000.000/0001-91,2019-03-31,1,BCO BRASIL SA,1023,DF Individual - Demonstração das Mutações do P...,REAL,MIL,2019-03-31,5.01,Saldo Inicial,6.700000e+07,S,2019-01-01,Capital Social Integralizado
